# Simplify SPMe+SEI Model 

The main purpose of this section is to reformulate the SPMe+RF model from. 

Assumptions:

<ol>
    <li> The diffusivity parameter of lithium ion in the negative particle is constant.</li>
    <li> The concentration of lithium ions in the electrolyte is constant.</li>
    <li> The conductivity and transport efficiency of the electrolyte are also constant </li>
    </ol>

$$   \frac{d \bar{\hat{c}}_{\mathrm{n0}}}{d\hat{t}}=\frac{-3}{\alpha_n }\hat{J}_\mathrm{n}^\mathrm{total}.$$

In [1]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt

In [2]:
cn = pybamm.Variable("Concentration [mol.m-3]")
phi_n = pybamm.Variable("Negative electrode potential [V]", domain="negative electrode")
phi_e_n = pybamm.Variable("Negative electrolyte potential [V]", domain="negative electrode")
E_n = pybamm.Variable("Negative electrode porosity", domain="negative electrode")
L_SEI = pybamm.Variable("Thickness of SEI [m]",  domain="negative electrode")

x_n = pybamm.SpatialVariable("x_n", domain=["negative electrode"], coord_sys="cartesian")

In [3]:
# dimensional parameters
k = pybamm.Parameter("Reaction rate constant [m.s-1]")
# c_inf = pybamm.Parameter("Bulk electrolyte solvent concentration [mol.m-3]")
R_n = pybamm.Parameter("Particle radius [m]")
#D = pybamm.FunctionParameter("Diffusion coefficient [m2.s-1]", {"Concentration [mol.m-3]": cn})
D = pybamm.Parameter("Diffusion coefficient [m2.s-1]")
#j = pybamm.InputParameter("Interfacial current density [A.m-2]")
T = pybamm.Parameter('Initial temperature [K]')
R = pybamm.Parameter("Molar gas constant [J.mol-1.K-1]")
F = pybamm.Parameter("Faraday constant [C.mol-1]")
cn0 = pybamm.Parameter("Initial concentration [mol.m-3]")
c_e = pybamm.Parameter("Electrolyte concentration [mol.m-3]")
L_n = pybamm.Parameter("Negative electrode thickness [m]")
I_app = pybamm.Parameter("Applied current [A]")
j_SEI_0 =  pybamm.Parameter('SEI reaction exchange current density [A.m-2]')
an = pybamm.Parameter('Particle surface area to volume ratio [m-1]')
U_SEI= pybamm.Parameter('SEI open-circuit potential [V]')
s_SEI = pybamm.Parameter("SEI conductivity [S.m-1]")
c_surf = pybamm.surf(cn)  # get the surface concentration
inputs = {"negative particle surface concentration [mol.m-3]": c_surf}
Un=pybamm.FunctionParameter("Negative electrode OCP [V]", inputs )
L_SEI_0 = pybamm.Parameter("Initial SEI thickness [m]")
alpha_SEI = pybamm.Scalar(.5);
sigma_e=pybamm.Parameter('Electrolyte conductivity [S.m-1]')
B = pybamm.Scalar(.5);
E_init=pybamm.Parameter('Initial negative electrode porosity')
M_SEI = pybamm.Parameter("Molar weight of SEI material [kg.mol-1]")
rho_SEI = pybamm.Parameter("Density of SEI material [kg.m-3]")
n_SEI = pybamm.Scalar(2)


In [4]:
from pybamm import tanh
from pybamm import exp

# # both functions will depend on the maximum concentration
# c_max = pybamm.Parameter("Maximum concentration in positive electrode [mol.m-3]")


# def exchange_current_density(c_surf):
#     k = 6 * 10 ** (-7)   # reaction rate [(A/m2)(m3/mol)**1.5]
#     c_e = 1000  # (constant) electrolyte concentration [mol.m-3]
#     return k * c_e** 0.5 * c_surf ** 0.5 * (c_max - c_surf) ** 0.5


def open_circuit_potential(xx):
#     stretch = 1.062
#     sto = stretch * c_surf / c_max

    u_eq = (
         0.2482
        + 1.9793*exp(-39.3631*xx)
        - 0.0909 * tanh(29.8538*(xx-0.1234))
        - 0.04478 * tanh(14.9159*(xx-0.2769))
        - 0.0205 * tanh(30.4444*(xx-0.6103))
    )
    return u_eq



In [5]:
#j_SR0 =  pybamm.FunctionParameter("Positive electrode exchange-current density [A.m-2]", inputs)
J_SR =  -an * j_SEI_0 * pybamm.exp(-alpha_SEI*F/(R*T) * (phi_n - phi_e_n - U_SEI- (I_app*L_SEI/(L_n*an*s_SEI))))
J_SR_AV=pybamm.x_average(J_SR);

In [6]:
model = pybamm.BaseModel()

model.algebraic = {
#      phi_n: I_app/L_n*x_n,
#     phi_n:  I_app/(2*sigma_e*B*L_n)*x_n*x_n,
#     phi_e_n: I_app/(2*sigma_e*B*L_n)*x_n*x_n,
    phi_n: Un,
    phi_e_n: I_app/(2*sigma_e*B*L_n)*x_n*x_n,
    L_SEI: L_SEI_0-1/an*(E_n-E_init),    
}

In [7]:
dcndt = -3/(an*R_n)*(I_app/L_n-J_SR_AV)
dEndt=M_SEI/(n_SEI*rho_SEI*F)*J_SR_AV

In [8]:
model.rhs = {cn: dcndt, E_n: dEndt} 

In [9]:
phi_n0= pybamm.Scalar(0.01)
phi_e_n0=pybamm.Scalar(-1.18485878)
L_SEI0=pybamm.Scalar(1e-9)
cn0=pybamm.Scalar(873)
En0=pybamm.Scalar(0.25)

In [10]:
# Initial Condition
# inputs = {"Initial concentration [mol.m-3]": cn0}
# U_init = pybamm.FunctionParameter("Negative electrode OCP [V]", inputs)
model.initial_conditions = {
    phi_n:phi_n0,
    phi_e_n: phi_e_n0,
    L_SEI: L_SEI0,
    cn : cn0,
    E_n : En0,
}

In [11]:
model.variables = {
    "Concentration [mol.m-3]": cn,
    "Negative electrode potential [V]": phi_n,
    "Negative electrolyte potential [V]": phi_e_n,
    "Negative electrode porosity": E_n, 
    "Thickness of SEI [m]": L_SEI,
}

In [12]:
geometry = {
    "negative electrode": {x_n: {"min": 0, "max": L_n}},
#     "negative particle": {r: {"min": 0, "max": R_n}},
}

In [13]:
param = pybamm.ParameterValues(
    {"Reaction rate constant [m.s-1]": 1e-6,
      "Initial SEI thickness [m]": 1e-12,
     "Particle radius [m]": 5.86e-06,
     "Diffusion coefficient [m2.s-1]": 3.3*1e-14,
     'Initial temperature [K]': 298,
     "Molar gas constant [J.mol-1.K-1]": 8.314,
     "Initial concentration [mol.m-3]": 1000,
     "Negative electrode thickness [m]": 8.52e-05,
     "Applied current [A]": 76,
     'SEI reaction exchange current density [A.m-2]': 1.5e-07,
     'Particle surface area to volume ratio [m-1]': 3.84 *1e5,
     'SEI open-circuit potential [V]': 0.4,
     "SEI conductivity [S.m-1]": 5e-6, 
      "Electrolyte conductivity [S.m-1]": 0.95,
     'Initial negative electrode porosity': 0.25,
     "Molar weight of SEI material [kg.mol-1]": 0.162,
     "Density of SEI material [kg.m-3]": 1690, 
      "Negative electrode OCP [V]": open_circuit_potential,
    }
)
    

In [14]:
param.process_model(model)
param.process_geometry(geometry)

In [15]:
submesh_types = {
    "negative electrode": pybamm.Uniform1DSubMesh,
#     "negative particle": pybamm.Uniform1DSubMesh,
}
# var_pts = {x_n: 20, r: 30}
var_pts = {x_n: 20}
mesh = pybamm.Mesh(geometry, submesh_types, var_pts)

spatial_methods = {
    "negative electrode": pybamm.FiniteVolume(),
#     "negative particle": pybamm.FiniteVolume(),
}
disc = pybamm.Discretisation(mesh, spatial_methods)
disc.process_model(model);

In [16]:
solver = pybamm.CasadiSolver()
t = np.linspace(0, 360, 100)
solution = solver.solve(model, t)

RuntimeError: .../casadi/core/function_internal.cpp:145: Error calling Newton::init for 'roots':
.../casadi/core/rootfinder.cpp:235: Assertion "!sp_jac_.is_singular()" failed:
Rootfinder::init: singularity - the jacobian is structurally rank-deficient. sprank(J)=0 (instead of 60)

In [ ]:
cycle_number =1
exp = pybamm.Experiment(
    [("Charge at C/10  until 4.2 V",
       "Rest for 10 hours",
     "Discharge at C/10 until 2.5 V",
     "Rest for 10 hours")] *cycle_number)
pybamm.lithium_ion.SPMe().variable_names()